## Virtual distillation with classical shadows 

Related paper [Seif et al, PRX Quantum 4, 010303 2023](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.4.010303)

In [1]:
using ITensors,ITensorMPS
using RandomMeas
using ProgressMeter

In [2]:
N = 4
ξ = siteinds("Qubit", N)
B = 1.
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising ZZ interactions
  ampo .+= -1, "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field X
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ)
H2 = apply(H,H)
# Density-matrix renormalization group
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = random_mps(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)
# Run 
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψ = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-4.7405782692369645  maxlinkdim=4 maxerr=7.72E-17 time=15.027
After sweep 2 energy=-4.758755919336972  maxlinkdim=4 maxerr=1.55E-17 time=0.005
After sweep 3 energy=-4.758770467729373  maxlinkdim=4 maxerr=3.60E-18 time=0.004
After sweep 4 energy=-4.758770483126646  maxlinkdim=4 maxerr=6.25E-18 time=0.004
After sweep 5 energy=-4.7587704831436115  maxlinkdim=4 maxerr=2.22E-16 time=0.017

Ground state energy:  -4.7587704831436115

---------------------------------------



In [3]:
#Consider experimental state admixed with white noise
p = 0.05
Ide = MPO(ξ)
for i in 1:N
    Ide[i] = δ(ξ[i]',ξ[i])
end
ρ = (1-p)*outer(ψ',ψ)+p*Ide/2^N;

In [4]:
nu = 500
NM = 100
data = zeros(Int8,(nu,NM,N));

In [5]:
Es = 0.
E2s = 0.
n = 3 #We will create three batch shadows
u = Vector{Vector{ITensor}}()
ITensors.disable_warn_order()
@showprogress dt=1 for r in 1:nu
        push!(u,get_rotations(ξ,1))
        data[r,:,:] =  get_RandomMeas(ρ,u[r],NM)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:28


In [6]:
ρb = get_batch_shadows(data,ξ,u,n)
ρm = Vector{ITensor}(undef,n)
ρm[1] = (ρb[1]+ρb[2]+ρb[3])/3/nu
ρm[2] = (multiply(ρb[1],ρb[2])+multiply(ρb[1],ρb[3])+multiply(ρb[2],ρb[3]))/3/nu
ρm[3] = multiply(multiply(ρb[1],ρb[2]),ρb[3])/nu;

In [7]:
Es = zeros(n)
for nt in 1:n
    Es[nt] = real(trace(multiply(ρm[nt],flatten(H)),ξ)/trace(ρm[nt],ξ))
end

In [8]:
println("Estimations for different purification order n " , Es)
println("Real values ",E)

Estimations for different purification order n [-4.601577317902415, -4.775973385643779, -4.74488219001357]
Real values -4.7587704831436115
